# Differential Accessibilty Analysis of Mutant vs Wild p53 

In [9]:
library(edgeR)
library(dplyr)

In [ ]:
setwd('data/')
can=c('BRCA','COAD')

In [11]:
for (val in can)
{
rawcounts <- as.matrix(read.csv(paste(val,'_rawcounts_MvsW.tsv',sep=""),sep="\t",row.names="Region"))
coldata <- read.csv(paste(val,'_condition_MvsW.tsv',sep=""),sep='\t', row.names='Sample', header=TRUE)

cpm_counts=cpm(rawcounts, normalized.lib.sizes=TRUE, log=FALSE, prior.count=5)
df2=cpm_counts[apply(cpm_counts[,1:ncol(cpm_counts)], 1, max) > 1,]
df2=rawcounts[rownames(df2),1:ncol(rawcounts)]

df=df2+5
snames<- colnames(df) # Sample names
group= coldata$Condition
mm <- model.matrix(~0 + group)
y <- voom(df, mm, normalize.method='quantile')
fit <- lmFit(y, mm)
contr <- makeContrasts(groupMutant - groupWild, levels = colnames(coef(fit)))
tmp <- contrasts.fit(fit, contr)
tmp <- eBayes(tmp)
limma_res <- topTable(tmp, sort.by = "P", n = Inf)
print(val)
print(sum(limma_res$adj.P.Val < 0.1 & abs(limma_res$logFC) >= 1))
write.table(limma_res, file=paste(paste('../Results/',val,sep=""),'_WM_results.tsv',sep=""),sep='\t')
}

[1] "BRCA"
[1] 2571
[1] "COAD"
[1] 1783
